# GEMlearner

> This package will hold the GxExM dataset.

In [ ]:
#| hide
from nbdev.showdoc import *
#| hide
from hybridpredictmaize22.GEMdataset import *

/home/burbank/miniconda3/envs/fastai/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from sklearn.preprocessing import StandardScaler
from pathlib import Path

from tqdm import tqdm

In [ ]:
def remove_leapdays(weather_data):
    obs = {}
    for env in weather_data['Env']:
        if env not in obs.keys():
            obs[env] = 1
        else:
            obs[env] += 1
            
    days = []
    for i in obs.values():
        for x in np.arange(1,i+1):
            days.append(x)
    weather_data['day'] = days
    
    return weather_data.drop(weather_data[weather_data['day'] == 366].index)

In [ ]:
path_snps = Path('data/snpCompress/')
data_path = Path('data/')

path_train_weatherTable =data_path/'Training_Data/4_Training_Weather_Data_2014_2021.csv'
path_train_yieldTable = data_path/'Training_Data/1_Training_Trait_Data_2014_2021.csv'

snp_data = collect_snps(path_snps/'EVEN_10') # Read in the SNP profiles
yield_data = pd.read_csv(path_train_yieldTable) # Read in trait data 
yield_data = yield_data[yield_data['Twt_kg_m3'].notnull()] #Remove plots w/ missing yields
weather_data = pd.read_csv(path_train_weatherTable) # Read in Weather Data
weather_data['Year'] = [x.split('_')[1] for x in weather_data['Env']] #Store Year in a new column
#removes yield data where no weather data
setYield = set(yield_data['Env'])
setWeather = set(weather_data['Env'])
only_yield = setYield - setWeather
only_weather = setWeather - setYield
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Env']],:]
#removes yield data where no genotype data
setSNP = set(snp_data[0])
setYield = set(yield_data['Hybrid'])
only_yield = setYield - setSNP
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Hybrid']],:]

weather_data = remove_leapdays(weather_data)

#Create a GEM dataset
test_split = 2019
gem = GEM(test_split)
gem.Y = YT(yield_data, test_split)
gem.W = WT(weather_data, test_split)
gem.SNP = collect_snps(path_snps/'PCS_10')

In [ ]:
tr_ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP)
te_ds = GemDataset(gem.W.Te, gem.Y.Te, gem.SNP)

In [ ]:

from sklearn.decomposition import PCA
pca = PCA(n_components=10)
pca.fit(gem.W.Tr.select_dtypes('float'))

xt = pca.transform(gem.W.Tr.select_dtypes('float'))

In [ ]:
len(gem.SNP[1][:,0])

20

In [ ]:
y,g,w = next(iter(tr_dataloader))

NameError: name 'tr_dataloader' is not defined

In [ ]:
y

In [ ]:
g

In [ ]:
w

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        
    def __call__(self,x):
        for l in self.layers:x = l(x)
        return x

In [ ]:
class Optimizer():
    def __init__(self, params, lr=0.5): self.params,self.lr=list(params),lr

    def step(self):
        with torch.no_grad():
            for p in self.params: p -= p.grad * self.lr

    def zero_grad(self):
        for p in self.params: p.grad.data.zero_()

In [ ]:
class gwModel(nn.Module):
    def __init__(self, gi,gh,wi,wh,c):
        super().__init__()
        self.gi = nn.Linear(gi,gh)
        self.wi = nn.Linear(wi,wh)

        self.gh = nn.Linear(gh,c)
        self.wh = nn.Linear(wh,c)
    
        self.o = nn.Linear(c*2,1)
    
    def forward(self,x):
        g,w = x
        
        w=w.flatten()
        
        #snp
        g = self.gi(g)
        g = torch.relu(g)
        g = self.gh(g)
        g = torch.relu(g)
        
        #wet
        w = self.wi(w)
        w = torch.relu(w)
        w = self.wh(w)
        w = torch.relu(w)
        
        #print(w.shape)
        g = (g.flatten())
        x = torch.cat((w,g), dim = 0)
        out = self.o(x)
        return out
        
       # print(x)

In [ ]:
gi = g.flatten().shape[0]
wi = w.flatten().shape[0]

gw = gwModel(gi,10,wi,10,5)

g=g.type(torch.float32)
w=w.type(torch.float32)
gw([g,w])

In [ ]:
gw = gwModel(gi,10,wi,10,5)
opt = Optimizer(gw.parameters())
loss_func = nn.MSELoss()

losses = []
for epoch in range(10):
    for batch in tqdm(range(1000)):
        y,g,w = next(iter(tr_dataloader))
        g=g.type(torch.float32)
        w=w.type(torch.float32)
        predict = gw([g,w])
        loss = loss_func(y.type(torch.float32),predict.squeeze().type(torch.float32))
        loss.backward()
        opt.step()
        opt.zero_grad()
        losses.append(loss)

In [ ]:
plt.plot()

In [ ]:
newis = []
for i in [x.detach().numpy() for x in losses]:
    if i > 0.1:
        pass
    else:
        newis.append(i)


In [ ]:

plt.plot([x.detach().numpy() for x in losses],)

In [ ]:
import torch
import torch.nn as nn

# Define the model
class MLP(nn.Module):
  def __init__(self, input_size1, input_size2, hidden_size, output_size):
    super().__init__()
    self.fc1 = nn.Linear(input_size1 + input_size2, hidden_size)
    self.fc2 = nn.Linear(hidden_size, output_size)

  def forward(self, x1, x2):
    # Concatenate the inputs
    x = torch.cat((x1, x2), dim=1)
    # Pass the concatenated inputs through the model
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.fc2(x)
    return x

# Create an instance of the model
model = MLP(input_size1=10, input_size2=5, hidden_size=20, output_size=1)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Train the model
for epoch in range(100):
  # Generate some fake data for training
  input1 = torch.randn(64, 10)
  input2 = torch.randn(64, 5)
  target = torch.randn(64, 1)

  # Forward pass
  output = model(input1, input2)
  loss = criterion(output, target)

  # Backward pass and optimization
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
wModel = Model(len(w[0].flatten()), 100, 1)
opt = Optimizer(wModel.parameters())
loss_func = nn.MSELoss()

In [ ]:
from torch.utils.data import DataLoader

bs = 1

tr_dataloader = DataLoader(tr_ds, batch_size=bs, shuffle=True)
te_dataloader = DataLoader(te_ds, batch_size=bs, shuffle=True)

In [ ]:
y,g,w = next(iter(tr_dataloader))

In [ ]:
y,g,w

In [ ]:
wModel = Model(len(w[0].flatten()), 2, 1)
opt = Optimizer(wModel.parameters())
loss_func = nn.MSELoss()
losses = []
for _ in range(1000):
    y,g,w = next(iter(tr_dataloader))
    xb = w.flatten().type(torch.float32)
    preds = wModel(xb)
    loss = loss_func(preds, y.type(torch.float32))
    losses.append(loss.detach().numpy())
    loss.backward()
    with torch.no_grad():
        for l in wModel.layers:
            if hasattr(l, 'weight'):
                l.weight -= l.weight.grad * lr
                l.bias   -= l.bias.grad   * lr
                l.weight.grad.zero_()
                l.bias  .grad.zero_()

In [ ]:
def f():
    yield from losses
    
from itertools import islice

In [ ]:
x = f()

In [ ]:
list(islice(x,5))

In [ ]:
plt.plot(losses)

In [ ]:
wModel = Model(len(w[0].flatten()), 2, 1)
opt = Optimizer(wModel.parameters())
loss_func = nn.MSELoss()

losses = []
for epoch in range(5):
    for batch in tqdm(range(100)):
        y,g,w = next(iter(tr_dataloader))
        weather_input = w.reshape(bs,w.shape[1]*w.shape[2]).type(torch.float32)
        predict = wModel(weather_input)
        loss = loss_func(y.type(torch.float32),predict.squeeze().type(torch.float32))
        loss.backward()
        opt.step()
        opt.zero_grad()
        losses.append(loss)

plt.plot([x.detach().numpy() for x in losses])

In [ ]:
gModel = Model(len(g[0].flatten()), 50, 1)
opt = Optimizer(gModel.parameters())
loss_func = nn.MSELoss()

losses = []
for epoch in range(5):
    for batch in tqdm(range(100)):
        y,g,w = next(iter(tr_dataloader))
        g_input = g.reshape(bs,g.shape[1]).type(torch.float32)
        predict = wModel(weather_input)
        loss = loss_func(y.type(torch.float32),predict.squeeze().type(torch.float32))
        loss.backward()
        opt.step()
        opt.zero_grad()
        losses.append(loss)

plt.plot([x.detach().numpy() for x in losses])

In [ ]:
y,g,w = next(iter(tr_dataloader))
weather_input = w.reshape(bs,w.shape[1]*w.shape[2]).type(torch.float32)
predict = wModel(weather_input)
loss = loss_func(y.type(torch.float32),predict.type(torch.float32))
loss.backward()
opt.step()
opt.zero_grad()
losses.append(loss)

In [ ]:
y.shape

In [ ]:
.shape

In [ ]:

y.shape

In [ ]:
g.shape

In [ ]:
w.shape

In [ ]:
wModel = Model(len(w[0].flatten()), 100, 1)

In [ ]:
w.shape

In [ ]:
weather_input = w.reshape(2,w.shape[1]*w.shape[2]).type(torch.float32)

In [ ]:
pred = wModel(weather_input)

In [ ]:
loss(pred,y)

In [ ]:
for target, inp in next(iter(tr_dataloader)):
    pass

In [ ]:
target.shape

In [ ]:
inp.shape

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()